In [1]:
import torch
import torchvision
import requests
from PIL import Image
from io import BytesIO
from matplotlib import pyplot as plt
import numpy as np
from diffusers import DiffusionPipeline

In [2]:
device = ("cuda:0,2,3" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0,2,3


In [3]:
def show_images(x):
    """Given a batch of images x, make a grid and convert to PIL"""
    x = x * 0.5 + 0.5  # Map from (-1, 1) back to (0, 1)
    grid = torchvision.utils.make_grid(x)
    grid_im = grid.detach().cpu().permute(1, 2, 0).clip(0, 1) * 255
    grid_im = Image.fromarray(np.array(grid_im).astype(np.uint8))
    return grid_im

def make_grid(images, size=64):
    """Given a list of PIL images, stack them together into a line for easy viewing"""
    output_im = Image.new("RGB", (size * len(images), size))
    for i, im in enumerate(images):
        output_im.paste(im.resize((size, size)), (i * size, 0))
    return output_im

In [4]:
pipeline = DiffusionPipeline.from_pretrained("stablediffusionapi/juggernaut-xl-v5")
pipeline.load_lora_weights("openskyml/dalle-3-xl")
if torch.cuda.device_count() > 1:
    pipeline = torch.nn.DataParallel(pipeline)
    
pipeline.to('cuda')

unet/diffusion_pytorch_model.safetensors not found


Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/home/aditi/miniconda3/envs/genai/lib/python3.11/site-packages/diffusers/loaders.py:1905: FutureWarning: `_modify_text_encoder` is deprecated and will be removed in version 0.25. You are using an old version of LoRA backend. This will be deprecated in the next releases in favor of PEFT make sure to install the latest PEFT and transformers packages in the future.
  deprecate("_modify_text_encoder", "0.25", LORA_DEPRECATION_MESSAGE)
/home/aditi/miniconda3/envs/genai/lib/python3.11/site-packages/diffusers/loaders.py:1876: FutureWarning: `_remove_text_encoder_monkey_patch_classmethod` is deprecated and will be removed in version 0.25. You are using an old version of LoRA backend. This will be deprecated in the next releases in favor of PEFT make sure to install the latest PEFT and transformers packages in the future.
  deprecate("_remove_text_encoder_monkey_patch_classmethod", "0.25", LORA_DEPRECATION_MESSAGE)


DataParallel()

In [11]:
generator = torch.Generator(device='cuda').manual_seed(42)

def segment_imagegen(segment_text):
  pipe_output = pipeline(
      prompt=segment_text, # What to generate
      generator=generator        # Fixed random seed
  )

  return pipe_output

In [12]:
segment_text = """A man and a woman long for a child. They have a window that overlooks a beautiful garden, but it belongs to an enchantress.
The woman sees rampion (rapunzel) in the garden and desires it, but cannot get it. She becomes sickly and her husband decides to get some for her"""

In [13]:
make_grid(segment_imagegen(segment_text).images,size=480)

AttributeError: 'StableDiffusionXLPipeline' object has no attribute 'parameters'